## Introduction

The goal of this analysis is to develop an image classification algorithm that can classify images into one of three categories: T-shirt/top, Sneaker, and Bag. By leveraging a subset of the Fashion MNIST dataset, we aim to build and evaluate a deep learning model using MobileNetV2 for this classification task.


## Data Source

The dataset used for this analysis is the Fashion MNIST dataset, which is publicly available and can be accessed through TensorFlow Datasets. Fashion MNIST consists of 70,000 grayscale images in 10 categories, with 7,000 images per category. For this analysis, we will use a subset of the dataset that includes three categories: T-shirt/top, Sneaker, and Bag. More information about the dataset can be found in Tensorflow_Datasets library.


## Import libraries

In [5]:
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import requests
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

: 

## Data Loading and Preprocessing

First, we load and normalize the Fashion MNIST dataset. We filter the dataset to include only three categories: T-shirt/top, Sneaker, and Bag. We also convert the grayscale images to RGB, resize them, and apply data augmentation techniques to enhance the training process.


In [3]:
(train_ds, validation_ds, test_ds), ds_info = tfds.load(
    "fashion_mnist",
    split=["train[:10%]", "train[10%:12%]", "train[12%:15%]"],
    as_supervised=True,  # Include labels
    with_info=True,
)
def filter_classes(image, label):
    return tf.math.logical_or(tf.math.equal(label, 0),
                              tf.math.logical_or(tf.math.equal(label, 7),
                                                 tf.math.equal(label, 8)))
train_ds = train_ds.filter(filter_classes)
validation_ds = validation_ds.filter(filter_classes)
test_ds = test_ds.filter(filter_classes)

# Normalize the images to [0, 1] range
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

train_ds = train_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
validation_ds = validation_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)


c:\Users\PARSA\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:04<00:01,  1.36s/ url]

Dl Completed...: 100%|██████████| 4/4 [00:12<00:00,  3.67s/ url]

Dl Completed...: 100%|██████████| 4/4 [00:12<00:00,  3.19s/ url]


Dataset fashion_mnist downloaded and prepared to C:\Users\PARSA\tensorflow_datasets\fashion_mnist\3.0.1. Subsequent calls will reuse this data.


## Dataset Preparation

Next, we resize the images to 224x224 pixels, which is the input size expected by the MobileNetV2 model. We also apply data augmentation techniques such as random horizontal flipping and rotation to enhance the model.

In [4]:
def convert_to_rgb(image, label):
    image = tf.image.grayscale_to_rgb(image)
    return image, label

train_ds = train_ds.map(convert_to_rgb, num_parallel_calls=tf.data.AUTOTUNE)
validation_ds = validation_ds.map(convert_to_rgb, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(convert_to_rgb, num_parallel_calls=tf.data.AUTOTUNE)

In [5]:
resize_fn = layers.Resizing(224, 224)

# Augmentation layers
augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(x, y):
    x = resize_fn(x)  # Add resizing inside data_augmentation
    for layer in augmentation_layers:
        x = layer(x)
    return x, y

train_ds = train_ds.map(data_augmentation, num_parallel_calls=tf.data.AUTOTUNE)

# Ensure validation and test datasets are resized
validation_ds = validation_ds.map(lambda x, y: (resize_fn(x), y), num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(lambda x, y: (resize_fn(x), y), num_parallel_calls=tf.data.AUTOTUNE)


In [6]:
from tensorflow import data as tf_data
batch_size = 64  

train_ds = train_ds.batch(batch_size).prefetch(1).cache()  # Prefetch 1 batch at a time
validation_ds = validation_ds.batch(batch_size).prefetch(1).cache()
test_ds = test_ds.batch(batch_size).prefetch(1).cache()

## Model Building and Training

We use the MobileNetV2 architecture, which is pre-trained on the ImageNet dataset. We customize the model to classify images into three categories: T-shirt/top, Sneaker, and Bag. We first train only the top layer of the model, keeping the pre-trained layers frozen. Then, we fine-tune the entire model with a low learning rate.

### Building the Model

In [7]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Build the model
inputs = tf.keras.Input(shape=(224, 224, 3))
scale_layer = layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(inputs)

x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)  
outputs = layers.Dense(10)(x)
model = tf.keras.Model(inputs, outputs)

model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


epochs = 5  
print("Fitting the top layer of the model")
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=validation_ds,
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Fitting the top layer of the model
Epoch 1/5
     28/Unknown 41s 1s/step - accuracy: 0.2917 - loss: 1.4301

C:\Users\PARSA\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


28/28 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.2923 - loss: 1.4244 - val_accuracy: 0.3585 - val_loss: 1.1094
Epoch 2/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.3365 - loss: 1.1706 - val_accuracy: 0.3585 - val_loss: 1.0851
Epoch 3/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.3174 - loss: 1.1710 - val_accuracy: 0.3585 - val_loss: 1.0775
Epoch 4/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.3568 - loss: 1.1337 - val_accuracy: 0.3585 - val_loss: 1.0570
Epoch 5/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.3637 - loss: 1.1117 - val_accuracy: 0.3585 - val_loss: 1.0441


After training the top layer, we proceed to fine-tune the entire MobileNetV2 model. Fine-tuning involves unfreezing the base model and training it end-to-end with a very low learning rate. This allows the weights of the pre-trained model to be slightly adjusted to better fit our specific dataset.

By fine-tuning, we aim to improve the model's performance by leveraging the rich features learned by the MobileNetV2 model during its pre-training on the ImageNet dataset, while adapting it to our specific task of classifying T-shirt/top, Sneaker, and Bag images from the Fashion MNIST dataset.


In [9]:
# Fine-tune the entire model
base_model.trainable = True
model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

print("Fitting the end-to-end model")
history_fine = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=validation_ds
)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 2,236,682 (8.53 MB)

 Non-trainable params: 34,112 (133.25 KB)

Fitting the end-to-end model
Epoch 1/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 222s 7s/step - accuracy: 0.3474 - loss: 1.9285 - val_accuracy: 0.3585 - val_loss: 1.0980
Epoch 2/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 182s 6s/step - accuracy: 0.7015 - loss: 0.6682 - val_accuracy: 0.5714 - val_loss: 1.1059
Epoch 3/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 167s 6s/step - accuracy: 0.9046 - loss: 0.3143 - val_accuracy: 0.3558 - val_loss: 1.1550
Epoch 4/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 179s 6s/step - accuracy: 0.9439 - loss: 0.2171 - val_accuracy: 0.3558 - val_loss: 1.3091
Epoch 5/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 174s 6s/step - accuracy: 0.9614 - loss: 0.1675 - val_accuracy: 0.3558 - val_loss: 1.4460


### Interpretation of Results

From the fine-tuning results, we observe that the training accuracy improved significantly, reaching over 96%, which is an excellent result. However, the validation accuracy did not improve correspondingly and remained around 35.58%. This discrepancy indicates that while the model performs exceptionally well on the training data, it does not generalize as well to the validation data.

In conclusion, the model shows strong learning capability on the training data, but additional efforts are needed to improve its performance on unseen data.
